In [1]:
# Import relevant packages
import os
import pandas as pd
import numpy as np

In [2]:
os.getcwd()

'C:\\Users\\robur\\Documents\\Thesis\\code\\lcfMapping'

In [3]:
# Read in excel files
pred2015 = pd.read_csv("../data/output/wurChange/predictions-2015-ndvi-median.csv")
pred2016 = pd.read_csv("../data/output/wurChange/predictions-2016-ndvi-median.csv")
pred2017 = pd.read_csv("../data/output/wurChange/predictions-2017-ndvi-median.csv")
pred2018 = pd.read_csv("../data/output/wurChange/predictions-2018-ndvi-median.csv")
classes = list(pred2015.columns)

pred2015

,tree,shrub,grassland,crops,urban_built_up,bare,water
0,0.000000,3.571429,96.428571,0.000000,0.0,0.000000,0.000000
1,61.764706,0.000000,38.235294,0.000000,0.0,0.000000,0.000000
2,0.000000,17.142857,65.714286,0.000000,0.0,17.142857,0.000000
3,0.000000,63.636364,9.090909,0.000000,0.0,27.272727,0.000000
4,34.285714,5.714286,60.000000,0.000000,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...
30484,100.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
30485,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,100.000000
30486,0.000000,0.000000,1.369863,0.000000,0.0,0.000000,98.630137
30487,0.000000,0.000000,100.000000,0.000000,0.0,0.000000,0.000000


In [4]:
for i in range(len(classes)):
    print(classes[i])

tree
shrub
grassland
crops
urban_built_up
bare
water


In [4]:
print("2015:", pred2015["tree"][1])
print("2016:", pred2016["tree"][1])
print("2017:", pred2017["tree"][1])
print("2018:", pred2018["tree"][1])

2015: 61.7647058823529
2016: 51.6129032258064
2017: 20.0
2018: 20.0


In [5]:
tree = pd.concat([pred2015["tree"],pred2016["tree"],pred2017["tree"],pred2018["tree"]], axis=1)
tree.columns = ['2015', '2016', '2017', '2018']
tree = tree.iloc[1]
tree

2015    61.764706
2016    51.612903
2017    20.000000
2018    20.000000
Name: 1, dtype: float64

In [ ]:
## Start of Markov chain building

In [85]:
def initialise(lcClass=None):
    
    node = lcClass[0]
    print("Node:", node)
    edgeUnary = tree[0]
    edgeNextYear = tree[1]
    
    return node, edgeUnary, edgeNextYear

def node_energy2015(node):
    
    # Squared-difference
    energyUnary = (node - edgeUnary)**2
    energyNextYear = (node - edgeNextYear)**2

    #print(energyUnary)
    #print(energyNextYear)

    # Energy
    energyTotal = energyUnary + energyNextYear
    #print("Total energy:", energyTotal)
    
    return(energyTotal)

In [84]:
# Intialise elements
start_node, edgeUnary, edgeNextYear =  initialise(tree)
energyTotal = node_energy2015(start_node)

# Create empty list to keep track of energy
energySim = []
energySim.append(energyTotal)

# Counter
print("Initial Energy:", energySim[i])

# Start loop
print("---")
print("Start simulation...")
node, energySim = minimise_energy(start_node, energySim)
print("Energy:", energySim)
print("Final node:", node)

Node: 61.7647058823529
Initial Energy: 103.0590971774646
---
Start simulation...
51.54107061199959
Here
2 100.02304404653458
Made it
Energy: [103.0590971774646, 51.54107061199959, 100.02304404653458, 82.32664935962758, 68.63025467272058, 58.93385998581358, 53.237465298906585, 51.54107061199959, 53.84467592509258]
Final node: 57.7647058823529


In [83]:
start_node
node_energy(node+1)
node_energy(node+1) < energySim[2-1]

False

In [82]:
def minimise_energy(node, energySim):
    
    i = 0
    
    if node > 50:
        node = node - 5
        e = node_energy2015(node)
        print(e)
        energySim.append(e)
        i += 1

        while e < energySim[i-1]:
            print("Here")
            node = node - 5
            e = node_energy(node)
            energySim.append(e)
            i += 1
            print(i, e)
        
        if node_energy(node+1) < energySim[i]:
            print("Made it")
            node = node + 1
            e = node_energy(node)
            energySim.append(e)
            i += 1
            
            while e < energySim[i-1]:
                node = node + 1
                e = node_energy(node)
                energySim.append(e)
                i += 1
        
    else:
        node = node + 5
        e = node_energy(node)
        energySim.append(e)
        i += 1

        while e < energySim[i]:
            node = node + 5
            e = node_energy(node)
            energySim.append(e)
            i += 1
            
        if node_energy(node-1) < energySim[i]:
            print("Made it")
            node = node - 1
            e = node_energy(node)
            energySim.append(e)
            i += 1
            
            while e < energySim[i-1]:
                node = node - 1
                e = node_energy(node)
                energySim.append(e)
                i += 1

    return node, energySim

In [95]:
# Initialise 2016 node and edges

node = tree[1]
print(node)

edgeUnary = tree[1]
print(edgeUnary)
edgeNextYear = tree[2]
print(edgeNextYear)


51.6129032258064
51.6129032258064
20.0


In [96]:
# Describe edges of 2016

# Actual difference
#print(node - edgeUnary)
#print(node - edgeNextYear)

# Squared-difference
energyUnary = (node - edgeUnary)**2
energyNextYear = (node - edgeNextYear)**2


#print(energyUnary)
#print(energyNextYear)

# Energy
energySim = []
energyTotal = energyUnary + energyNextYear
print("Total energy:", energyTotal)
energySim.append(energyTotal)

# Counter
i = 0

Total energy: 999.3756503642006


In [97]:
energySim[i]


999.3756503642006

In [99]:
if node > 50:
    node = node - 5
    e = node_energy(node)
    #print(e)
    energySim.append(e)
    i += 1
    
    while e < energySim[i-1]:
        print(e, energySim[i-1])
        node = node - 5
        e = node_energy(node)
        energySim.append(e)
        i += 1
    
else:
    node = node + 5
    e = node_energy(node)
    energySim.append(e)
    i += 1
    
    while e < energySim[i]:
        node = node + 5
        e = node_energy(node)
        energySim.append(e)
        i += 1
    
#node
print(energySim)
print(node)

Total energy: 500.9885535900086
[999.3756503642006, 733.2466181061366, 567.1175858480726, 500.9885535900086, 534.8595213319446, 500.9885535900086]
36.6129032258064


999.3756503642006

In [67]:
def node_energy(node):
    
    # Squared-difference
    energyUnary = (node - edgeUnary)**2
    energyNextYear = (node - edgeNextYear)**2

    #print(energyUnary)
    #print(energyNextYear)

    # Energy
    energyTotal = energyUnary + energyNextYear
    print("Total energy:", energyTotal)
    
    return(energyTotal)